# Usage example of VAR-Information decomposition analysis

In [ ]:
import numpy as np
import os
from NuMIT.NuMIT import NuMIT 
from utils.utils import loadData, saveData

## This procedure calculates PhiID atoms across the whole brain, sampling random brain regions, calculating PhiID between them, and repeating the process many times

In [ ]:
# set the path of the data
path = os.path.expanduser("~")+"/../../data/aliardi/psychedelics_MEG_600Hz"

# set drug name, number of channels, redundancy function, condition (drug/placebo)
dr = "LSD"
ch = 10 
rf = "Broja"
cond = "Drug"

# set the subject info
subject_info = {"drug": dr}
subject_info['name'] = 1
subject_info["condition"] = cond

# load the data
data, subject_info  = PsychMEGLoad(subject_info, path)

print(f"Doing drug {dr}, subject {subject_info['name']} in condition {subject_info['name']}.")
out_dir = f"Results/PhiID/{dr}/s{subject_info['name']}_{cond}/{rf}_c{ch}_results.pickle"

# set the parameters
params = {"channels": ch, "trials": 50, "runs": 100, 
        "red_fun": rf, "method": "PhiID", "mmorder": 1}

# run the analysis
results = VAR_analysis(data, subject_info, params, on_file=True, verbose=False, overwrite=False , path="Results/")

## Alternatively, use the VAR procedure on specific regions as follows

In [28]:
from VAR_NuMIT import PhiID_VAR_calculator
from VAR_fitness import fit_var
from utils import saveData, low_pass_filter


# set the path of the data
path = os.path.expanduser("~")+"/../../data/aliardi/psychedelics_MEG_600Hz"

# set drug name, number of channels, redundancy function, condition (drug/placebo)
dr = "LSD"
ch = 10 
rf = "Broja"
cond = "Placebo"
trials = 50 

# set the subject info
subject_info = {"drug": dr}
subject_info['name'] = 2 
subject_info["condition"] = cond

# load the data
data, subject_info  = PsychMEGLoad(subject_info, path)
n_chans, n_timepoints, n_trials = data.shape

# Downsample and filter the data
down_s = subject_info['fs'] // 200
ds_data = low_pass_filter(data, down_s, subject_info['fs'])

# Do PhiID between channels 1-5 and 5-10
channels = np.arange(ch)
# Sample 'trials' (e.g. 50) random trials for fitting the VAR model
trials = np.random.choice(n_trials, trials, replace=False)
# Select the channels and the trials
ts = ds_data[channels,:,:][:,:,trials]

# Fit VAR model
A, V, p, _ = fit_var(ts, maxp=1) #NB: VAR-PhiID only works with maxp=1

# Compute atoms from VAR model
atoms = PhiID_VAR_calculator(p, A, V, ch//2, ch//2, rf, verbose=True, as_dict=True)[0] # Set as_dict to False to obtain an array of atoms

print(atoms)

Optimizing with Adam...
Broja double union is 2.1556470562489265
{'rtr': 0.70249, 'rta': 0.0262, 'rtb': 0.03796, 'rts': -0.16239, 'xtr': 0.03796, 'xta': 0.85476, 'xtb': -0.03796, 'xts': 0.19099, 'ytr': 0.0262, 'yta': -0.0262, 'ytb': 0.53423, 'yts': 0.20278, 'str': -0.16695, 'sta': 0.19111, 'stb': 0.20765, 'sts': 0.10273}
